In [144]:
!pip install pyspark

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pyparsing/core.py", line 4114, in parseImpl
    return e._parse(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pyparsing/core.py", line 821, in _parseNoCache
    loc, tokens = self.parseImpl(instring, pre_loc, doActions)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pyparsing/core.py", line 2341, in parseImpl
    raise ParseException(instring, loc, self.errmsg, self)
pip._vendor.pyparsing.exceptions.Parse

In [145]:
import pyspark as ps
import pandas as pd

spark = ps.sql.SparkSession.builder.master("local").appName("test1").getOrCreate()
data = [("Alice", 25), ("Bob", 30), ("Charlie", 22)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)
df.show()

+-------+---+
|   Name|Age|
+-------+---+
|  Alice| 25|
|    Bob| 30|
|Charlie| 22|
+-------+---+



In [146]:
# spark.stop()

In [147]:
spark2 = ps.sql.SparkSession.builder.getOrCreate()
book_df = spark2.read.csv('books.csv',header = True,inferSchema=True)
book_df.show(10)

+---+--------+------------+--------+-----------+----------+----------------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+--------------------+--------------------+
| id| book_id|best_book_id| work_id|books_count|      isbn|          isbn13|             authors|original_publication_year|      original_title|               title|language_code|average_rating|ratings_count|work_ratings_count|work_text_reviews_count|ratings_1|ratings_2|ratings_3|ratings_4|ratings_5|           image_url|     small_image_url|
+---+--------+------------+--------+-----------+----------+----------------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+-----------

In [148]:
book_df.count(),len(book_df.columns)

(10000, 23)

In [149]:
book_df.describe("books_count").show(5)

+-------+-----------------+
|summary|      books_count|
+-------+-----------------+
|  count|            10000|
|   mean|          75.7127|
| stddev|170.4707276502584|
|    min|                1|
|    max|             3455|
+-------+-----------------+



In [150]:
book_df.select('original_title').show(5)

+--------------------+
|      original_title|
+--------------------+
|    The Hunger Games|
|Harry Potter and ...|
|            Twilight|
|To Kill a Mocking...|
|    The Great Gatsby|
+--------------------+
only showing top 5 rows



In [151]:
book_df.groupBy('authors').count().show()

+--------------------+-----+
|             authors|count|
+--------------------+-----+
|         Lisa Genova|    4|
|          James Frey|    3|
|              Banksy|    1|
|Grant Morrison, D...|    1|
|    Elin Hilderbrand|   15|
|     Karen Armstrong|    1|
|Thomas More, Paul...|    1|
|        Mia Sheridan|    2|
|         Kylie Scott|    5|
|      Rosamund Hodge|    1|
|Garth Ennis, Stev...|    1|
|Isa Chandra Mosko...|    1|
|          Dava Sobel|    1|
|Fábio Moon, Gabri...|    1|
|   Sharon Kay Penman|    3|
|    Joshilyn Jackson|    3|
|        Geoff Colvin|    1|
|Maud Hart Lovelac...|    1|
|         Bruce Degen|    1|
|        Doug Stanton|    1|
+--------------------+-----+
only showing top 20 rows



In [152]:
book_df.filter('average_rating > 4.5').show(10)

+---+--------+------------+--------+-----------+----------+----------------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+--------------------+--------------------+
| id| book_id|best_book_id| work_id|books_count|      isbn|          isbn13|             authors|original_publication_year|      original_title|               title|language_code|average_rating|ratings_count|work_ratings_count|work_text_reviews_count|ratings_1|ratings_2|ratings_3|ratings_4|ratings_5|           image_url|     small_image_url|
+---+--------+------------+--------+-----------+----------+----------------+--------------------+-------------------------+--------------------+--------------------+-------------+--------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+-----------

In [153]:
book_df.select('authors').distinct().count()

4664

In [154]:
df2 = book_df.dropDuplicates()
df2 = df2.dropna()
df2.count()

7860

In [155]:
df2.withColumn('average_rating',df2.average_rating / 5 * 100).show(5)

+---+-------+------------+-------+-----------+---------+----------------+--------------------+-------------------------+--------------------+--------------------+-------------+-----------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+--------------------+--------------------+
| id|book_id|best_book_id|work_id|books_count|     isbn|          isbn13|             authors|original_publication_year|      original_title|               title|language_code|   average_rating|ratings_count|work_ratings_count|work_text_reviews_count|ratings_1|ratings_2|ratings_3|ratings_4|ratings_5|           image_url|     small_image_url|
+---+-------+------------+-------+-----------+---------+----------------+--------------------+-------------------------+--------------------+--------------------+-------------+-----------------+-------------+------------------+-----------------------+---------+---------+---------+---------+---------+-----------

### Pyspark with machinelearning(Classification)

In [156]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [158]:
spark = SparkSession.builder.appName("IrisClassification").getOrCreate()

In [159]:
data = spark.read.csv("iris.csv", header=True, inferSchema=True)
data.show(10)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
|         5.4|        3.9|         1.7|        0.4| setosa|
|         4.6|        3.4|         1.4|        0.3| setosa|
|         5.0|        3.4|         1.5|        0.2| setosa|
|         4.4|        2.9|         1.4|        0.2| setosa|
|         4.9|        3.1|         1.5|        0.1| setosa|
+------------+-----------+------------+-----------+-------+
only showing top 10 rows



In [160]:
# Converting 'species' column to numerical
indexer = StringIndexer(inputCol="species", outputCol="label")
data = indexer.fit(data).transform(data)
data.show(5)

+------------+-----------+------------+-----------+-------+-----+
|sepal_length|sepal_width|petal_length|petal_width|species|label|
+------------+-----------+------------+-----------+-------+-----+
|         5.1|        3.5|         1.4|        0.2| setosa|  0.0|
|         4.9|        3.0|         1.4|        0.2| setosa|  0.0|
|         4.7|        3.2|         1.3|        0.2| setosa|  0.0|
|         4.6|        3.1|         1.5|        0.2| setosa|  0.0|
|         5.0|        3.6|         1.4|        0.2| setosa|  0.0|
+------------+-----------+------------+-----------+-------+-----+
only showing top 5 rows



In [161]:
# Assembling features into a single vector
feature_cols = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
data = assembler.transform(data)
data.show(5)

+------------+-----------+------------+-----------+-------+-----+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|species|label|         features|
+------------+-----------+------------+-----------+-------+-----+-----------------+
|         5.1|        3.5|         1.4|        0.2| setosa|  0.0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| setosa|  0.0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| setosa|  0.0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| setosa|  0.0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| setosa|  0.0|[5.0,3.6,1.4,0.2]|
+------------+-----------+------------+-----------+-------+-----+-----------------+
only showing top 5 rows



In [162]:
train_df, test_df = data.randomSplit([0.8, 0.2], seed=42)

### RandomForest

In [163]:
rf_classifier = RandomForestClassifier(featuresCol="features", labelCol="label")
model = rf_classifier.fit(train_df)

In [164]:
# predictions
pred = model.transform(test_df)

In [165]:
#evaluation
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 95.83%


In [166]:
result = pred.select("prediction", "label", "features")
result.show()

+----------+-----+-----------------+
|prediction|label|         features|
+----------+-----+-----------------+
|       0.0|  0.0|[4.4,3.0,1.3,0.2]|
|       0.0|  0.0|[4.6,3.2,1.4,0.2]|
|       0.0|  0.0|[4.6,3.6,1.0,0.2]|
|       0.0|  0.0|[4.8,3.1,1.6,0.2]|
|       0.0|  0.0|[4.9,3.1,1.5,0.1]|
|       1.0|  1.0|[5.0,2.3,3.3,1.0]|
|       0.0|  0.0|[5.0,3.5,1.3,0.3]|
|       0.0|  0.0|[5.1,3.5,1.4,0.2]|
|       0.0|  0.0|[5.3,3.7,1.5,0.2]|
|       1.0|  1.0|[5.4,3.0,4.5,1.5]|
|       0.0|  0.0|[5.4,3.4,1.5,0.4]|
|       0.0|  0.0|[5.4,3.7,1.5,0.2]|
|       0.0|  0.0|[5.4,3.9,1.7,0.4]|
|       1.0|  1.0|[5.5,2.5,4.0,1.3]|
|       1.0|  1.0|[5.6,2.9,3.6,1.3]|
|       1.0|  1.0|[5.7,2.9,4.2,1.3]|
|       2.0|  2.0|[5.8,2.7,5.1,1.9]|
|       2.0|  1.0|[6.3,2.5,4.9,1.5]|
|       2.0|  2.0|[6.4,3.1,5.5,1.8]|
|       2.0|  2.0|[6.5,3.0,5.2,2.0]|
+----------+-----+-----------------+
only showing top 20 rows



### NaiveBayes

In [167]:
nb_classifier = NaiveBayes(featuresCol="features", labelCol="label")

nb_model = nb_classifier.fit(train_df)

nb_predictions = nb_model.transform(test_df)

nb_result = nb_predictions.select("prediction", "label", "features")

nb_result.show()

+----------+-----+-----------------+
|prediction|label|         features|
+----------+-----+-----------------+
|       0.0|  0.0|[4.4,3.0,1.3,0.2]|
|       0.0|  0.0|[4.6,3.2,1.4,0.2]|
|       0.0|  0.0|[4.6,3.6,1.0,0.2]|
|       0.0|  0.0|[4.8,3.1,1.6,0.2]|
|       0.0|  0.0|[4.9,3.1,1.5,0.1]|
|       1.0|  1.0|[5.0,2.3,3.3,1.0]|
|       0.0|  0.0|[5.0,3.5,1.3,0.3]|
|       0.0|  0.0|[5.1,3.5,1.4,0.2]|
|       0.0|  0.0|[5.3,3.7,1.5,0.2]|
|       1.0|  1.0|[5.4,3.0,4.5,1.5]|
|       0.0|  0.0|[5.4,3.4,1.5,0.4]|
|       0.0|  0.0|[5.4,3.7,1.5,0.2]|
|       0.0|  0.0|[5.4,3.9,1.7,0.4]|
|       1.0|  1.0|[5.5,2.5,4.0,1.3]|
|       1.0|  1.0|[5.6,2.9,3.6,1.3]|
|       1.0|  1.0|[5.7,2.9,4.2,1.3]|
|       2.0|  2.0|[5.8,2.7,5.1,1.9]|
|       1.0|  1.0|[6.3,2.5,4.9,1.5]|
|       2.0|  2.0|[6.4,3.1,5.5,1.8]|
|       2.0|  2.0|[6.5,3.0,5.2,2.0]|
+----------+-----+-----------------+
only showing top 20 rows



In [168]:
# Evaluation
nb_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
nb_accuracy = nb_evaluator.evaluate(nb_predictions)
print("Naive Bayes Accuracy: {:.2f}%".format(nb_accuracy * 100))

Naive Bayes Accuracy: 100.00%
